In [7]:
from longeval.spark import get_spark
from longeval.collection import ParquetCollection
from pyspark.sql import functions as F

spark = get_spark()
root = "../../tests/integration"
collection = ParquetCollection(spark, f"{root}/parquet/train/2023_01/English")
collection.queries.printSchema()
collection.queries.count()

25/03/15 08:05:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


root
 |-- qid: string (nullable = true)
 |-- query: string (nullable = true)



599

In [5]:
row = collection.queries.sample(False, 0.1, 42).head()
print(row)

collection.qrels.printSchema()
df = collection.qrels.where(F.col("qid") == row.qid)
df.count(), df.distinct().count()
relevant_set = {row.docid for row in df.collect() if row.rel > 0}
relevant_set

Row(qid='q0123805', query='Veal filet mignon')
root
 |-- qid: string (nullable = true)
 |-- rank: integer (nullable = true)
 |-- docid: string (nullable = true)
 |-- rel: integer (nullable = true)



{'doc012303512545',
 'doc012304816815',
 'doc012305205106',
 'doc012307703830',
 'doc012308900581',
 'doc012312602366'}

In [6]:
from opensearchpy import OpenSearch
from sklearn.metrics import ndcg_score

client = OpenSearch("http://localhost:9200")
client.indices.get(index="train-english-2023_01")

query = row.query
results = client.search(
    index="train-english-2023_01",
    body={"query": {"match": {"contents": query}}},
    size=1000,
)
docids = [hit["_source"]["docid"] for hit in results["hits"]["hits"]]
print(docids[:10])
true_relevance = [1 if docid in relevant_set else 0 for docid in docids]
predicted_relevance = [1] * len(docids)
ndcg_score([true_relevance], [predicted_relevance]), sum(true_relevance)

['doc012300800010', 'doc012306300006']


(0.0, 0)